# Convert Barwar lexicon

In [1]:
import os
import re

from html_to_nena import html_elements, Text, element_totext, normalize_styles, text_tostring, str_replace

In [2]:
text_dir = 'texts'
basename = 'bar glossary general'
html_ext = '.html'
text_ext = '.txt'

# # Replace substrings provided in `replace`
# if replace:
#     s = str_replace(s, replace)
replace = {
    # standardizing substutions
    '\u2011': '\u002d',  # non-breaking hyphen to normal hyphen-minus
    '\u01dd': '\u0259',  # 'ǝ' "turned e" to 'ə' schwa
    '\uf1ea': '\u003d',  # SIL deprecated double hyphen to '=' equals sign
    '\u2026': '...',  # '…' horizontal ellipsis to three dots
    'J\u0335': '\u0248',  # 'J' + short combining stroke to 'Ɉ' J with stroke
    'J\u0336': '\u0248',  # J' + long combining stroke to 'Ɉ' J with stroke
    '<y>': '\u02b8',  # superscript small letter y
    # special corrections for lexicon
    '*\u02b8*': '\u02b8',  # superscript small letter y in italic context
    '<|>': '\u02c8',  # superscript vertical bar
    '*\u02c8 *': '\u02c8 ',  # superscript vertical bar in italic text
    '*\u02c8': '\u02c8*',  # superscript vertical bar after italic text
    ' |** ': '** | ',  # vertical bar accidentally marked bold
    ' **|** ': ' | ',  # vertical bar accidentally marked bold
    ' *| ': ' | *',  # vertical bar accidentally marked italic
    ' | (': ' (',  # superfluous vertical bar before numbered translations
    '**; **': '; ', # unmarked punctuation between marked text
    '*, *': ', ', # unmarked punctuation between marked text
    '**.**': '.', # dot accidentally marked bold
}
filename = '../texts/email Khan 2019-06-28/bar glossary general.html'

In [3]:
# these are the strings that can make up the grammatical categories of words
# in a non-capturing group (starting with ?:) of alternatives, sorted longest first
grm_cats = '(?:abs|and|adj|adv|cst|cs|fpl|fs|f|imper|interj|invar|mod|ms|m|num|n|part|pl|prep|pron|sing|tant|tan)'

languages = '(?:A|E|K|P|Ṭiy|Ṭiy|T|Urm|C.Syr)'

def parselemma(s):
        
    # first try to strip off numbered list of meanings: (1) ... (2) ... etc
    result = re.split(' (\(\d+\)) ', s)
    # if not found, try other pattern for numbered list: 1. ... 2. ... etc
    # (in Barwar lexicon: only for one lemma: 'kapora')
    if len(result) == 1:
        result = re.split(' (\d+\.) ', s)
    # if there are any meanings, recombine with their numbers in list of strings;
    # otherwise, empty list
    meanings = [f'{result[i]} {result[i+1]}' for i in range(1, len(result), 2)]
    # reassign s to part of string before numbered meanings (if any)
    s = result[0]
    
    # then try to strip off any examples, separated from the lemma by '|'
    result = re.split('\s*\|\s*', s, 1)
    # examples will be empty string if there are none
    examples = result[1] if len(result) > 1 else ''
    # reassign s to part of string before examples (if any)
    s = result[0]
    
    # split on the first bold marker `**` to get the translation.
    # since bold and italic text messes up the markers,
    # cannot just select from start to end.
    result = re.split('\s*(\*\*)\s*', s, 1)
    if len(result) > 1:
        # small fix for when opening brackets of translation aren't marked bold
        if result[0].endswith(' ('):
            result[0] = result[0][:-2]
            result[2] = '(' + result[2]
        trans = result[1] + result[2]
    else:
        trans = ''
    s = result[0]
    
    # split on ' → ' for references
    result = re.split('\s*→\s*', s, 1)
    # reference will be empty string if there is none
    reference = result[1] if len(result) > 1 else ''
    # reassign s to part of string before reference (if any)
    s = result[0]
    
    # now, we will continue by looking at the start of the string for the lemma head
    result = re.split('^\*([^*]+)\*\s*', s)
    if len(result) == 1:
        raise ValueError('No lemma at start of string:', s)
    lemma = result[1]
    s = result[2]
    
    # some lemmata have a roman numeral following it (though it seems unnecessary)
    result = re.split('^\(([ivx]+)\)\s*', s)
    numeral = result[1] if len(result) > 1 else ''
    # last element of result contains remaining string, whether numeral is found or not
    s = result[-1]
    
    # now s should start with a grammatical category of a word, such as 'n.f.'.
    # remove grammatical categories from the start of the string until there are no more
#     grm_desc = ''
#     while s:
#         result = re.split(f'^({grm_cats}(?:[ /.,]+|$))', s)
#         if len(result) == 1:
#             break
#         # if there is a match, element 0 is always empty (since match is from start of string);
#         # element 1 contains the match, element 2 the remainder of the string
#         grm_desc += result[1]
#         s = result[2]
#     grm_desc = grm_desc.rstrip()
    # like above, but with repeated non-capturing group instead of while loop
    result = re.split(f'^((?:{grm_cats}(?:[.,/ ]+|$))+)', s)
    # element 0 is always empty (bcs of '^' in regex); if match, element 1 contains match
    grm_desc = result[1].rstrip() if len(result) > 1 else ''
    # last element of result contains remaining string, whether match is found or not
    s = result[-1]
    
    # split on round brackets to get annotations for lemma.
    # even elements in position >= 2 should be empty
    # e.g.: '*word* n.m. (pl. *words*) (E.)' results in:
    # ['*word* n.m.', 'pl. *words*', '', 'E.', '']
    result = re.split('\s*\(([^)]+)\)\s*', s)
    # list of annotations, if any, else empty
    annotations = [result[i] for i in range(1, len(result), 2)]
    # list of things in between (should all be empty strings! just checking!)
    inbetweens = [result[i] for i in range(2, len(result), 2) if result[i]]
    # everything after lemma and before annotations is the tail
    tail = result[0]
    
    # extract forms from annotations
    resultlist = []
    forms = []
    for e in annotations:
        # split word forms in format 'f. *worda, wordb*, pl. *wordc*' from start of string
        result = re.split(fr'^((?:(?:{grm_cats}\.?\s+)+\*[^*]+\*(?:[,;]?\s|$))+)', e)
        match = result[1] if len(result) > 1 else None
        resultlist.append(result[-1])
        if match:
            # split matched string into seperate word forms and their grammatical decribers
            result = re.split(fr'((?:{grm_cats}\.?\s+)+)\*([^*]+)\*(?:[,;]?\s|$)', match)
            # the above example results in:
            # ['', 'f.', 'worda, wordb', '', 'pl.', 'ʾwordc', '']
            # The first regex selecting the whole string makes sure that nothing can occur
            # in the elements 0, 3, 6 etc., so we can safely skip those
            forms += [(result[i].rstrip(), result[i+1]) for i in range(1, len(result), 3)]
    annotations = [e for e in resultlist if e]

    # extract languages from annotations
    resultlist = []
    lang = ''
    for e in annotations:
        # make sure that only one language string is returned
        if lang:
            resultlist.append(e)
            continue 
        result = re.split(fr'^((?:{languages}\.)(?:/{languages}\.)*)', e)
        if not lang and len(result) > 1:
            lang = result[1]
            break 
        resultlist.append(result[-1])
    annotations = [e for e in resultlist if e]
    
    # TODO most stuff is extracted. Some stuff however does not fit the regexes.
    # (Remains of) unconventional notations can be seen in the tail, annotations
    # and (especially) the inbetweens variables. Some of these may require manual
    # corrections, others may be parsed by more sophisticated regexes.
    # (e.g.: alternative forms in the notation: 'sing. *wordform* f.')
    
    # TODO Both `meanings` and `examples` are mostly structured to some degree.
    # They could be parsed further (e.g. `meanings` usually contain examples).
    
    # TODO The `grm_desc` should be parsed further, now they are just strings,
    # e.g. 'n.m.' for the head word, and 'f.' for an alternative form.
    # It should be clearer that the head word is e.g. noun, masc, sing and that
    # the alternative form is e.g. noun, fem, sing.
    
    parsed = {
        'lemma': lemma,
        'numeral': numeral,
        'grm_desc': grm_desc,
        'forms': forms,
        'trans': trans,
        'tail': tail,
        'lang': lang,
        'annotations': annotations,
        'inbetweens': inbetweens,
        'ref': reference,
        'examples': examples,
        'meanings': meanings,
    }
    
    return parsed

In [4]:
filename = os.path.join(text_dir, basename + html_ext)

lemmata = []

for e in html_elements(filename):
    t = Text(element_totext(e))
    t = normalize_styles(t, can_have_emphasis=lambda c: not c.isspace())
    s = text_tostring(t)
    s = str_replace(s, replace)

    if not s or s.isdigit():
        continue
    elif not s.startswith('*'):
        continue
        if s.startswith('/'):
            print('##', s)
        else:
            print('#', s)
    else:
        parsed = parselemma(s)
        lemma_str = '\n'.join(f'{k}: {v!r}' for k, v in parsed.items() if v)
        
        lemmata.append(lemma_str)
        
        print(lemma_str)
        print()

lemma: 'ʾabaya, ʾabbaya'
grm_desc: 'n.m.'
forms: [('pl.', 'ʾabaye, ʾabbaye')]
trans: '**man’s cloak**'
lang: 'A.'
examples: '*ʾabàyele də́rya b-réšaˈ* He put his cloak over her (A26:50); *ʾɛ́-ga kúlla ʾaġăwáθa šexìyeˈ kúlla b-ʾabbàye-wawaˈ* At that time all aghas and elders wore the abbaya gown (A26:47); *ʾíman ṱ-ila-nabólle bə̀draˈ m-gu-ʾằra,ˈ mattíwa ... ʾabàya y-amrə́xwaleˈ* When they took it (the rice) to the threshing floor from that ground, they would lay down what we called a cloak (B5:80).'

lemma: 'ʾabona'
grm_desc: 'n.m.'
forms: [('pl.', 'ʾabone')]
trans: '**bishop**'
lang: 'A.'

lemma: 'ʾabresəm'
ref: '*habresəm*'

lemma: 'ʾăbu'
grm_desc: 'part.'
trans: '**attributive particle** (A. literally: father of)'
examples: '*ʾăbu-bə̀rqaˈ* electrician(s) (B10:50); *băyánni ʾə́mma bnàθaˈ ṱ-áwa ʾăbu tā̀jˈ* I want for myself a hundred girls wearing crowns (A25:74). In expressions of age: *ʾo-yála zòraˈ ʾábu ʾəsrì-šənneˈ* The youth, who was twenty years old (A17:2).'

l

lemma: 'ʾaxala'
grm_desc: 'n.m./adj.'
forms: [('pl.', 'ʾaxale')]
trans: '**big eater**'

lemma: 'ʾaxalta'
grm_desc: 'n.f./adj.'
forms: [('pl.', 'ʾaxalyaθa')]
trans: '**big eater (f.)**'

lemma: 'ʾaxči'
grm_desc: 'part.'
trans: '**just, only; but.**'
meanings: ['(1) **just, only** | *xúwwe ʾáxčile múdwa rìqaˈ* The snake just managed to escape (A1:15); *ʾáxči ʾáp ʾayya-béna múrri mòdileˈ* Just one more time tell me what it is (A1:18); *malàxe yắði,ˈ ʾàxčiˈ* Only the angels know (A4:9); *ʾáxči xoš-qàyəmˈ xazyánne xàmša daqíqe,ˈ ʾan-xéne kúlla ṭla-d-àwˈ* Just let him rise for me to see him for five minutes, and all the other (sixty years) be for him (A4:49).', '(2) **but** | *ʾə́č̣č̣i-u ʾə̀č̣č̣a,ˈ ʾáxči ʾána max-xšàwti,ˈ ʾáyya kìstaˈ hóle mxožə́bnəlla max-xàˈ* (There are) ninety-nine (coins), but according my opinion, he has reckoned this bag as one (A6:3)']

lemma: 'ʾaxḍar'
grm_desc: 'adj. invar.'
trans: '**green**'
lang: 'A.'

lemma: 'ʾaxəl-qarṣa'
grm_desc: 'n.

lemma: 'ʾiða'
grm_desc: 'n.f.'
forms: [('pl.', 'ʾiθaθa, ʾiðăwaθa ʾiðe')]
trans: '**hand; handle; arm; foot (of an animal); round of a game.**'
meanings: ['(1) **hand** | *ʾíðux lá-darət gu-nùraˈ t-là-ʾaqðaˈ* Do not put your hand in the fire, lest it burn (D2:8); *b-ʾíðe díye dawùqleˈ* He would hold it in his hand (B6:3); *zríyənwale b-iθàθiˈ* I had cultivated it with my hands (A17:32); *b-ìðən xɛṭə́xwaˈ* We would sew by hand (B10:59); *hállule l-ʾìðiˈ* Hand him over into my custody (A28:25).', '(2) **handle** | *ʾiðət tăra* handle of a door; *ʾiðət magreta* the handle of the shaving knife.', '(3) **arm** | *dawɛ́ra ʾíðe-w ʾàqleˈ* A mule (is a man’s) arms and legs (B5:124); *šə́ryəlla ʾáqle w-ìðeˈ* He untied the legs and arms (A10:13).', '(4) **foot (of an animal)** | *qímle tàwraˈ primále ðá ʾíðe dìyeˈ* He cut off one of the feet of the ox (A22:3).', '(5) **round of a game** | *kəmà ʾiθáθa mṭáləx?ˈ* How many rounds shall we play?; *ṭḷà-ʾiθaθa muxsə́rre Mắmoˈ* Mă

lemma: 'barəbar–'
trans: '**around**'
lang: 'K.'
examples: 'inside an area: *ʾu-y-azíwa xaðrìwaˈ barəbárət màθaˈ* They would go around the village (B15:66)'

lemma: 'bărəkθa'
grm_desc: 'n.f.'
trans: '**(small) sheet**'
lang: 'K.'
examples: '*pšúṭla ʾàqluxˈ qắdər bărə̀kθuxˈ* Stretch your legs according to your cloth (D1:6)'

lemma: 'bar-eli'
grm_desc: 'adv.'
trans: '**upwards, on the upper side**'
examples: '*m-pálgə d-áw ʾu-bàr-eliˈ* from its middle and upwards (A8:58); *múrməlla ʾaqláθa l-bàr-eliˈ ʾu-réša xtàyaˈ* They lifted her legs upwards and her head downwards (A27:34).'

lemma: 'barəšta, berəšta'
grm_desc: 'n.f.'
forms: [('pl.', 'barəšyaθa, berəšyaθa')]
trans: '**pillow, cushion**'

lemma: 'barəxmaya'
grm_desc: 'n.m.'
forms: [('pl.', 'barəxmaye')]
trans: '**brother of wife**'

lemma: 'barəxmɛθa'
grm_desc: 'n.f.'
forms: [('pl.', 'barəxmayaθa')]
trans: '**sister of wife**'

lemma: 'băriya'
grm_desc: 'n.f.'
trans: '**creation, creatures**'
lang: 'A.'

lemma: 'bariyya, 

lemma: 'bəðya'
grm_desc: 'adj.'
forms: [('f.', 'bðiθa'), ('pl.', 'bəðye')]
trans: '**luxuriant (plant)**'
examples: '*xa ṭaqət rixana bəðya* (She is like) a sprig of luxuriant basil (C2:72).'

lemma: 'bəjbaja'
grm_desc: 'n.m.'
forms: [('pl.', 'bəjbaje')]
trans: '**insect**'
examples: '*bəjbajət ʾəxrət tawre* dung beetle'

lemma: 'bəlād'
grm_desc: 'n.f.'
trans: '**country**'
lang: 'A.'

lemma: 'bəlbala'
grm_desc: 'n.m.'
trans: '**chaos**'
examples: '*wírre bəlbàla gawéy* They were in chaos.'

lemma: 'bəlbəlawa'
grm_desc: 'n.m.'
trans: '**celebrations at the end of winter**'
examples: '*xaðríwa bəlbəlàwa* We would go around (performing) *bəlbəlawa* (i.e. children go around the village asking for gifts B7:7).'

lemma: 'bəlbəllizək'
grm_desc: 'n.f.'
trans: '**whirlwind, hurricane**'
lang: 'K.'

lemma: 'bəldozer'
grm_desc: 'n.m.'
forms: [('pl.', 'bəldozere')]
trans: '**bulldozer**'
lang: 'A./E.'

lemma: 'bəllorta'
grm_desc: 'n.f.'
forms: [('pl.', 'bəlloryaθa')]
trans: '**pipe; section 

...

lemma: 't-la'
grm_desc: 'part.'
trans: '**without**'
examples: 'With nominal complement: *t-la-zùzeˈ* without money (A24:23); *t-lá mìyaˈ* without water (B5:14); *t-lá d-o-náša* without that man; *t-la-díyi* without me. With an infinitive: *zeðàyeˈ ʾáxlət mə́nna t-là-qṣayaˈ* You should eat the pittas without breaking them. (A30:5); *ʾáni xə́ṭṭe huwèdi sáma zódaˈ ʾi-zăríla t-là maštóyeˈ* The *huwedi* wheat they generally sow without irrigating (B5:94); *qímɛle jálde madə́nxa t-la-mxàyaˈ* He rose early while the dawn was not breaking (= before dawn broke) (A8:51); *qəm-dári ʾan-dáwwe heš-t-là brázaˈ* They have poured out the *dawwe* without it yet being dry (B14:1).'

lemma: 'tmanəssər, tmanessər'
grm_desc: 'num.'
trans: '**eighteen**'

lemma: 'tmani'
grm_desc: 'num.'
trans: '**eighty**'

lemma: 'tmanya'
grm_desc: 'num.'
forms: [('f.', 'tmane')]
trans: '**eight**'
examples: '*ʾurzət tmanya* the eighth man; *baxtət tmane* the eighth woman.'

lemma: 'tmanyaθn–'
grm_d

lemma: 'ṭḷaθn–'
grm_desc: 'num.'
trans: '**three of**'
examples: 'base for pronominal suffixes: *ṭḷaθnən* the three of us; *ṭḷaθnɛxu* the three of you; *ṭḷaθnéy* the three of them; *ṭḷáθna t-bnóne dìyeˈ* His three sons (A24:3)'

lemma: 'ṭḷaθušeba'
grm_desc: 'n.m.'
trans: '**Tuesday**'

lemma: 'ṭliba'
grm_desc: 'n.m.'
forms: [('pl.', 'ṭlibe')]
trans: '**fiancé, betrothed**'

lemma: 'ṭlibta'
grm_desc: 'n.f.'
forms: [('pl.', 'ṭlibyaθa')]
trans: '**fiancée, betrothed (f.)**'

lemma: 'ṭliya'
grm_desc: 'adj.'
forms: [('f.', 'ṭlita'), ('pl.', 'ṭliye')]
trans: '**asleep**'

lemma: 'ṭolaza'
grm_desc: 'n.m.'
forms: [('pl.', 'ṭolaze')]
trans: '**young man; womanizer**'
lang: 'K.'
examples: '*šəṭro ṭolaze baθra npile* (My) beautiful one (lover)—young men followed after her (C2:30).'

lemma: 'ṭōq'
grm_desc: 'n.m.'
trans: '**necklace**'
lang: 'A.'

lemma: 'ṭoriya, ṭoriθa'
grm_desc: 'n.f.'
forms: [('pl.', 'ṭoriyaθa')]
trans: '**jackal**'
lang: 'K.'

lemma: 'ṭoṭiθa'
grm_desc: 'f.'
forms: [('pl.', 

lemma: 'xanpa'
grm_desc: 'n.m.'
forms: [('pl.', 'xanpe')]
trans: '**pagan, heathen**'
examples: '*ʾu-Dùreˈ píštɛla muqyàmtaˈ muxràwtaˈ šawwà gáyeˈ l-xànpeˈ qaṭòleˈ t-mušəlmàneˈ* Dure was evacuated and destroyed seven times by pagan murders of Muslims (B2:2).'

lemma: 'xanqa-xona'
grm_desc: 'n.m.'
forms: [('pl.', 'xanqa-xone')]
trans: '**small white flower**'

lemma: 'xanum'
grm_desc: 'n.f.'
trans: '**lady**'
lang: 'K.'
examples: '*dax-xànum ʾay-tíwta gu-bɛ́θaˈ* Like a lady she sits at home (A37:3).'

lemma: 'xanuθa'
grm_desc: 'n.f.'
forms: [('pl.', 'xanăwaθa, xanuyaθa')]
trans: '**loom for weaving**'

lemma: 'xapa'
grm_desc: 'n.m.'
forms: [('pl.', 'xape')]
trans: '**wooden peg used to connect the main body of a plough to the beam (***mašana***)**'

lemma: 'xapaya'
grm_desc: 'adj.'
forms: [('f.', 'xapɛθa'), ('pl.', 'xapaye')]
trans: '**protecting, caring**'
examples: '*báxta xapɛ̀θaˈ* a woman is caring (A40:2)'

lemma: 'xapro'
grm_desc: 'n.f.'
forms: [('pl.', 'xaprăwaθa')]
t

lemma: 'xəmyana'
grm_desc: 'n.m.'
trans: '**father-in-law**'

lemma: 'xənne'
grm_desc: 'adj.f.'
trans: '**chestnut coloured (goat)**'

lemma: 'xəppərta'
grm_desc: 'n.f.'
forms: [('pl.', 'xəppəryaθa')]
trans: '**mud pit**'
examples: 'where white clay for the facing of houses is excavated'

lemma: 'xəppo'
grm_desc: 'n.f.'
forms: [('pl.', 'xəppowaθa')]
trans: '**bridal veil**'

lemma: 'xəpyaya'
grm_desc: 'adj.'
forms: [('f.', 'xəpyɛθa'), ('pl.', 'xəpyaye')]
trans: '**barefoot**'

lemma: 'xərba'
grm_desc: 'adj. invar.'
trans: '**bad**'

lemma: 'xərbayuθa'
grm_desc: 'n.f.'
trans: '**evil**'

lemma: 'xərqa'
grm_desc: 'n.m.'
trans: '**ring**'
examples: 'in neck cartilage: *băláwala yabíšta xərqá xərqá gu-qðála díya mabyə̀nwaˈ* When she swallowed a raisin, it could be seen, ring by ring in her neck (A8:63); *xərqət ʾilana* growth-ring in tree; *xərqət qana* growth-ring in horn.'

lemma: 'xərroza'
grm_desc: 'n.m.'
forms: [('pl.', 'xərroze')]
trans: '**string of dried fruit (e.g. figs, dat

lemma: 'yarəkθa'
grm_desc: 'n.f.'
forms: [('pl.', 'yarəkyaθa')]
trans: '**concubine, mistress**'
lang: 'K.'

lemma: 'yarqa'
grm_desc: 'n.m.'
trans: '**vegetables; vegetable stew**'
examples: '*ṱ-oðánna yàrqa-lleyˈ* I shall make it into a vegetable stew for them (A34:18).'

lemma: 'yarxa, yɛrxa'
grm_desc: 'n.m.'
forms: [('pl.', 'yarxe, yɛrxe')]
trans: '**month**'
examples: '*təltàssər b-yɛ́rxaˈ* the thirteenth of the month (B16:22); *yarxət tre* February; *yarxət ʾəč̣č̣a* September; *rešət yarxa* the beginning of the month; *ʾítla réšət yàrxaˈ* She has a menstrual period.'

lemma: 'yaṣila, yaṣilta'
grm_desc: 'n.m.'
forms: [('pl.', 'yaṣile, yaṣilyaθa')]
trans: '**outer edge (of wall, window); outer corner (of building); side (of door)**'
examples: '*kút yaṣílət qàṣraˈ ʾíθ ða-yàwna gáwe díyeˈ* On every corner of the palace there is a dove (A12:24); *rúše díye qíθɛle b-yaṣílət tằraˈ* His shoulder hit the side of the door (A29:10). Cf. *zawiθa* inner corner'

lemma: 'yasmi

In [5]:
filename_txt = os.path.join(text_dir, basename + text_ext)
with open(filename_txt, 'w') as f:
    f.write('\n\n'.join(lemmata))